In [10]:
import openai
from openai import OpenAI
from dotenv import load_dotenv
import os
from PIL import Image
from io import BytesIO

load_dotenv() 

client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  
)

# set a directory to save DALL·E images to
image_dir = "../static/img"


In [11]:

def downsample_image_to_buffer(input_path, max_size_bytes=4*1024*1024, step=10, resize_factor=0.9):
    """
    Downsample the image and return a BytesIO object within the specified size limit.
    
    Args:
        input_path (str): Path to the input image.
        max_size_bytes (int): Maximum allowed size in bytes.
        step (int): Step size for reducing quality.
        resize_factor (float): Factor to reduce dimensions by in each iteration.
        
    Returns:
        BytesIO: A file-like object containing the downsampled image.
    """
    img = Image.open(input_path)
    img_format = img.format  # Preserve the original image's format
    quality = 95
    
    img_bytes = BytesIO()
    while True:
        img_bytes.seek(0)  # Reset the buffer pointer to the beginning
        img.save(img_bytes, format=img_format, quality=quality, optimize=True)
        file_size = img_bytes.tell()  # Get the current size of the buffer
        
        if file_size <= max_size_bytes:
            break  # The image is now under the threshold
        
        if quality > step:
            quality -= step
        else:
            img = img.resize((int(img.width * resize_factor), int(img.height * resize_factor)), Image.ANTIALIAS)
    
    img_bytes.seek(0)  # Reset buffer pointer to the beginning for reading
    return img_bytes

In [12]:
# For example(s):
# https://github.com/openai/openai-cookbook/blob/main/examples/dalle/Image_generations_edits_and_variations_with_DALL-E.ipynb

# This buffer be used directly with client.images.create_variation or similar functions
variation_response = client.images.create_variation(
    image=downsample_image_to_buffer(os.path.join(image_dir, "img_2_202403030949.png")),
    n=1,
    size="512x512",
    response_format="url",
)

variation_response